In [1]:
import logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)
import torch
import torch.nn.functional as F

from german_parser.util.const import CONSTS
from german_parser.util.dataloader import TigerDatasetGenerator
import dill as pickle
from string import punctuation
from german_parser.model import TigerModel
from german_parser.util import BatchUnionFind

from german_parser.util.c_and_d import Dependency, DependencyTree, ConstituentTree, Terminal
from collections import defaultdict
from typing import Sequence

In [2]:
(train_dataloader, train_new_words), (dev_dataloader, dev_new_words), (test_dataloader, test_new_words), character_set, character_flag_generators, inverse_word_dict, inverse_sym_dict = pickle.load(open("required_vars.pkl", "rb"))
model: TigerModel = pickle.load(open("./models/epoch_25_cpu_eval.pickle", "rb"))
dl_en = enumerate(test_dataloader)

In [3]:
model.cuda()

TigerModel(
  (word_embedding): WordEmbedding(
    (word_cnn): WordCNN(
      (embeddings): Embedding(111, 100, padding_idx=1)
      (conv): Conv1d(105, 100, kernel_size=(3,), stride=(1,))
    )
    (embeddings): Embedding(25019, 100, padding_idx=1)
  )
  (enc_lstm): LSTMSkip(
    (chain): ModuleList(
      (0): LSTM(200, 512, batch_first=True, bidirectional=True)
      (1-2): 2 x LSTM(1024, 512, batch_first=True, bidirectional=True)
    )
    (dropout): Dropout(p=0.2, inplace=False)
  )
  (dec_lstm): LSTMSkip(
    (chain): ModuleList(
      (0): LSTM(1024, 512, batch_first=True)
    )
    (dropout): Dropout(p=0, inplace=False)
  )
  (enc_final_cell_to_dec_init_cell): Linear(in_features=1024, out_features=512, bias=True)
  (enc_attention_mlp): Linear(in_features=1024, out_features=512, bias=True)
  (dec_attention_mlp): Linear(in_features=512, out_features=512, bias=True)
  (enc_label_mlp): Linear(in_features=1024, out_features=128, bias=True)
  (dec_label_mlp): Linear(in_features=512, 

In [12]:
from typing import Any


def f1_evaluator(dataloader: Any, out_dir="./eval/"):
    brackets: list[str] = []
    t_brackets: list[str] = []

    brackets_structure: list[str] = []
    t_brackets_structure: list[str] = []

    for i, (x, l, target_ex, target_lab, target_att) in enumerate(test_dataloader):

        x = x.cuda()
        l = l.cpu()
        best_edges, labels_best_edges, attachment_orders_best_edges, (edges, joint_logits) = model.find_tree((x, l), test_new_words)

        print(i)

        for s_num in range(x.shape[0]):
            words = x[s_num, :l[s_num]].to("cpu")

            ex = best_edges[s_num, :l[s_num]].to("cpu")
            lab = labels_best_edges[s_num, :l[s_num]].to("cpu")
            att = attachment_orders_best_edges[s_num, :l[s_num]].to("cpu")

            t_ex = target_ex[s_num, :l[s_num]].to("cpu")
            t_lab = target_lab[s_num, :l[s_num]].to("cpu")
            t_att = target_att[s_num, :l[s_num]].to("cpu")

            the_sentence = [inverse_word_dict[w.item()] if w > 0 else test_new_words[-w.item()] for w in words]

            c_tree = ConstituentTree.from_collection(heads=ex, syms=[inverse_sym_dict[l.item()] for l in lab], orders=att, words=the_sentence)
            t_c_tree = ConstituentTree.from_collection(heads=t_ex, syms=[inverse_sym_dict[l.item()] for l in t_lab], orders=t_att, words=the_sentence)

            brackets.append(c_tree.get_bracket(zero_indexed=True, ignore_words=True))
            brackets_structure.append(c_tree.get_bracket(ignore_all_syms=True, zero_indexed=True, ignore_words=True))

            t_brackets.append(t_c_tree.get_bracket(zero_indexed=True, ignore_words=True))
            t_brackets_structure.append(t_c_tree.get_bracket(ignore_all_syms=True, zero_indexed=True, ignore_words=True))

    open(f"{out_dir}/b.txt", "w").write("\n".join(brackets))
    open(f"{out_dir}/t_b.txt", "w").write("\n".join(t_brackets))

    open(f"{out_dir}/b_s.txt", "w").write("\n".join(brackets_structure))
    open(f"{out_dir}/t_b_s.txt", "w").write("\n".join(t_brackets_structure))

In [15]:
f1_evaluator(dev_dataloader)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
